In [200]:
import konlpy
import nltk
from nltk import Tree, pos_tag, word_tokenize, RegexpParser 
from nltk.draw.util import CanvasFrame
from nltk.draw import TreeWidget
from nltk.draw.tree import TreeView
from nltk import CFG

import os

import re

from bokeh.io import export_png
import threading

from koalanlp import API
from koalanlp.proc import Parser

In [38]:
nltk.download("averaged_perceptron_tagger")
nltk.download('punkt') 

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Samsung\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Samsung\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
data = ['코드리스 이어폰은 기어 x 2018 이후로 두 번째 입니', '다. 기어 x를 쓰다가 외이도 염 때문에 커 널 형은 쓰지 말라고', '해서 1년 간 오픈 형 유선 이어폰으로 연명하다가 싼 맛에 쓸 만 하다는 소문을 듣고 구입해 봤습니다.', '제가 쓰는 유선 이어폰은 갤 s6 의 번들 이어폰인 eg920 입니다.', '1. 가격 일단 가격이 깡패라', '조금 덜 소중히 다뤄도 된다는 게 제일 만족스럽습니다.', '제가 쓰던 유선 이어폰의 정가가 22000원으로 기억하는데 그것보다 저렴하니까요. 그래도 파우치에 넣어 가지고 다니 긴 하지만 주머니에서 굴리다가 이어폰이 빠질까 봐 불안해서 넣습니다.', '가격이 저렴해서 망가져도 as는 딱히 받을 생각이 없지만 초기 불량에 대한 불안감이 커서 정식 제품으로 샀는데, 이 또한 만족스럽습니다.', '물론 초기 불량이 나 진 않았지만요. 2. 통화 품질 시도도 안 해봤습니다.', 't5를 쓰는 친구와 종종 통화를 하는데, 바람이 많이 불면 친구의 목소리가 아예 안 들리는 걸 경험했고, 이건 그것보다 더 별로라는 얘기를 들어서 통화할 땐 그냥 유선 이어폰을 끼고 합니다.', '인도어에서는 해도 괜찮긴 하겠네요.', '3. 음질 음질은 제가 막 귀라', '그냥저냥 듣긴 하는데 생각보다 더 구립니다.', '제가 웬만하면 음질의 차이를 구별하지 못하는데, 이건 구별이 되는 정도 니 까요.', '이게 t1 만 그런 건지, 아니면 다른 저가 형 코드리스도 그런지는 잘 모르겠네요.', '제 폰이 갤 럭 시 s8 이라, 이전에 썼던 기어 x는 삼성 폰이랑 연결하면 알아서 음질이 좋아지는 것 같은 효과를 내주는 소프트웨어? 코덱? 이 적용돼서 불편함을 느낀 적은 없던 걸로 기억하는데, 그런 요행이 없으면 대부분의 저가 형 코드리스는 비슷한 수준일 것 같긴 합니다.', '가격이 비슷한 데 좋으면 얼마나 더 좋겠어요.', '4. 영상과의 싱크 유 튜브는 불편함 없이 시청이 가능했고, 폰의 비디오 플레이어도 딱히 딜레이는 못 느꼈는데( 블루 레이 같은 고용량의 파일은 안 해봐서 모르겠습니다.)', '공 단기 플레이어로 인터넷 강의를 들을 때는 딜레이가 느껴지네요.', '이건 어차피 강사의 입을 볼게', '아니라서 저한 텐 상관없습니다.', '배속을 올리면 체감하기도 힘들고요.', '5. 음량 유일하게 불만인 점인데, 음량 자체가 너무 작습니다.', '제가 쓰던 유선 이어폰은 오픈 형 임에도 불구하고 이거보단 훨씬 커요.', '많은 차들이 빨리 달리는 도로 옆을 걷다 보면 거의 안 들리는 정도라서 그런 도로는 피해 다닙니다.', '독서실에서 강의를 들을 때도 맥시멈 인 15로 하고 들어야 잘 들리는 수준이라, 6~7 이면 충분한 유선 이어폰이랑 차이가 큽니다.', '6. 배터리 하루종일 끼고 있는 게 아니고, 한번 낄 때 끽해야 한 시간 정도만 끼고 있어서 아직 까지 부족함은 잘 모르겠습니다.', '스펙 상 명시된 연속 사용시간이 좀 적은 것 같긴 하던데, 어차피 귀가 아파서 그렇게 오래는 못 끼고 있는지라 저한테 는 충분합니다.', '초기 불량으로 인한 교환가능 시기는 지났고, 가격이 가격인 만큼 3개월만 안 망가지고 버텨 주면 그 이후에 망가져도 재구매할 것 같습니다.', '귀에 염증이 생긴 뒤로 코드리스에는 관심도 없다가 t1x 가 나온다 기에 다시 관심을 갖기 시작했는데, 아직은 정식 수입이 되는 것 같지 않길래 아쉬운 대로 t1 을 산 거라서, t1x 도 정식수입을 해 주셨으면 좋겠습니다!']

In [12]:
words = konlpy.tag.Twitter().pos(data[0])

In [13]:
words

[('코드', 'Noun'),
 ('리스', 'Noun'),
 ('이어폰', 'Noun'),
 ('은', 'Josa'),
 ('기어', 'Noun'),
 ('x', 'Alpha'),
 ('2018', 'Number'),
 ('이후', 'Noun'),
 ('로', 'Josa'),
 ('두', 'Noun'),
 ('번째', 'Suffix'),
 ('입니', 'Adjective')]

In [ ]:
grammar = """
NP: {<N.*>*<Suffix>?}   # Noun phrase
VP: {<V.*>*}            # Verb phrase
AP: {<A.*>*}            # Adjective phrase
"""
parser = nltk.RegexpParser(grammar)
chunks = parser.parse(words)
#chunks.draw()


In [9]:
cf = CanvasFrame()
tc = TreeWidget(cf.canvas(),chunks)
cf.add_widget(tc)

In [17]:
groucho_grammar = CFG.fromstring("""
S -> NP VP
 PP -> P NP
 NP -> Det N | Det N PP | 'I'
 VP -> V NP | VP PP
 Det -> 'an' | 'my'
 N -> 'elephant' | 'pajamas'
 V -> 'shot'
 P -> 'in'
 """)
parser = nltk.ChartParser(groucho_grammar)
sent = ['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas']
trees = parser.parse(sent)

In [30]:

cf = CanvasFrame()
t = Tree.fromstring('(S (NP this tree) (VP (V is) (AdjP pretty)))')
tc = TreeWidget(cf.canvas(),t)
cf.add_widget(tc,10,10) # (10,10) offsets
cf.print_to_file('tree.ps')
os.chdir(r"C:\Users\Samsung\Desktop\Project\이희정 교수님\project\parse_tree\test")
os.system('convert output.ps output.png')

4

In [33]:
cf = CanvasFrame()
t = Tree.fromstring('(S (NP this tree) (VP (V is) (AdjP pretty)))')
tc = TreeWidget(cf.canvas(),t)
cf.add_widget(tc,10,10) # (10,10) offsets
cf.print_to_file('tree.ps')

In [40]:
# Import required libraries 
   
# Example text 
sample_text = "The quick brown fox jumps over the lazy dog"
   
# Find all parts of speech in above sentence 
tagged = pos_tag(word_tokenize(sample_text)) 
   
#Extract all parts of speech from any text 
chunker = RegexpParser(""" 
                       NP: {<DT>?<JJ>*<NN>}    #To extract Noun Phrases 
                       P: {<IN>}               #To extract Prepositions 
                       V: {<V.*>}              #To extract Verbs 
                       PP: {<P> <NP>}          #To extract Prepostional Phrases 
                       VP: {<V> <NP|PP>*}      #To extarct Verb Phrases 
                       """) 
  
# Print all parts of speech in above sentence 
output = chunker.parse(tagged) 
print("After Extracting\n", output) 

After Extracting
 (S
  (NP The/DT quick/JJ brown/NN)
  (NP fox/NN)
  (VP (V jumps/VBZ) (PP (P over/IN) (NP the/DT lazy/JJ dog/NN))))


In [44]:
output.draw()

In [51]:
#words = konlpy.tag.Twitter().pos(data[2])
#words
words = konlpy.tag.Twitter().pos("TV 테스트 겸 넷 플릭스 재생 해봤는데 액션 신에서 거슬리지 않으면서 적당히 심장에 쿵쿵 울림을 주는 소리가 나옵니다.5.")

In [53]:
words

[('TV', 'Alpha'),
 ('테스트', 'Noun'),
 ('겸', 'Noun'),
 ('넷', 'Noun'),
 ('플릭스', 'Noun'),
 ('재생', 'Noun'),
 ('해봤는데', 'Verb'),
 ('액션', 'Noun'),
 ('신', 'Noun'),
 ('에서', 'Josa'),
 ('거슬리지', 'Verb'),
 ('않으면서', 'Verb'),
 ('적당히', 'Adjective'),
 ('심장', 'Noun'),
 ('에', 'Josa'),
 ('쿵쿵', 'Adverb'),
 ('울림', 'Noun'),
 ('을', 'Josa'),
 ('주는', 'Verb'),
 ('소리', 'Noun'),
 ('가', 'Josa'),
 ('나옵니다', 'Verb'),
 ('.', 'Punctuation'),
 ('5', 'Number'),
 ('.', 'Punctuation')]

In [52]:
chunker = RegexpParser(""" 
                       NP: {<DT>?<JJ>*<NN>}    #To extract Noun Phrases 
                       P: {<IN>}               #To extract Prepositions 
                       V: {<V.*>}              #To extract Verbs 
                       PP: {<P> <NP>}          #To extract Prepostional Phrases 
                       VP: {<V> <NP|PP>*}      #To extarct Verb Phrases 
                       """) 
output = chunker.parse(words) 
#print("After Extracting\n", output)
output.draw()

## grammer 별 tree 형태

In [54]:
grammer_v1 = """
NP: {<N.*>*<Suffix>?}   # Noun phrase
VP: {<V.*>*}            # Verb phrase
AP: {<A.*>*}            # Adjective phrase"""

grammer_v2 = """
NP: {<DT>?<JJ>*<NN>}    #To extract Noun Phrases 
P: {<IN>}               #To extract Prepositions 
V: {<V.*>}              #To extract Verbs 
PP: {<P> <NP>}          #To extract Prepostional Phrases 
VP: {<V> <NP|PP>*}      #To extarct Verb Phrases """

grammer_v3 = """
VP: {<ADJ_SIM><V_PRS>}
VP: {<ADJ_INO><V.*>}
VP: {<V_PRS><N_SING><V_SUB>}
NP: {<N_SING><ADJ.*><N_SING>}
NP: {<N.*><PRO>}
VP: {<N_SING><V_.*>}
VP: {<V.*>+}
NP: {<ADJ.*>?<N.*>+ <ADJ.*>?}
DNP: {<DET><NP>}
PP: {<ADJ_CMPR><P>}
PP: {<ADJ_SIM><P>}
PP: {<P><N_SING>}
PP: {<P>*}
DDNP: {<NP><DNP>}
NPP: {<PP><NP>+}"""

In [67]:
chunker = RegexpParser(grammer_v3) 
output = chunker.parse(words) 
#print("After Extracting\n", output)
output.draw()

## RegexpParser 형태를 Tree formating으로

In [60]:
tree = Tree("s",output)

In [62]:
print(tree)

(s
  TV/Alpha
  (NP 테스트/Noun 겸/Noun 넷/Noun 플릭스/Noun 재생/Noun)
  (VP 해봤는데/Verb)
  (NP 액션/Noun 신/Noun)
  에서/Josa
  (VP 거슬리지/Verb 않으면서/Verb)
  적당히/Adjective
  (NP 심장/Noun)
  에/Josa
  쿵쿵/Adverb
  (NP 울림/Noun)
  을/Josa
  (VP 주는/Verb)
  (NP 소리/Noun)
  가/Josa
  (VP 나옵니다/Verb)
  ./Punctuation
  (NP 5/Number)
  ./Punctuation)


In [63]:
cf = CanvasFrame()
tree = Tree("s",output)
tc = TreeWidget(cf.canvas(),t)
cf.add_widget(tc,10,10) # (10,10) offsets


In [ ]:
cf.print_to_file('tree.ps')

## nltk.Tree 함수

In [213]:
dp1 = Tree('dp', [Tree('d', ['the']), Tree('np', ['dog'])])
dp2 = Tree('dp', [Tree('d', ['the']), Tree('np', ['cat'])])
vp = Tree('vp', [Tree('v', ['chased']), dp2])
tree = Tree('s', [dp1, vp])
print(tree)


(s (dp (d the) (np dog)) (vp (v chased) (dp (d the) (np cat))))


In [214]:
chunker = RegexpParser(grammer_v3) 
output_2 = chunker.parse(tree)
print(output_2)

(s (dp (d the) (np dog)) (vp (v chased) (dp (d the) (np cat))))


In [215]:
tree.pretty_print()

              s               
      ________|_____           
     |              vp        
     |         _____|___       
     dp       |         dp    
  ___|___     |      ___|___   
 d       np   v     d       np
 |       |    |     |       |  
the     dog chased the     cat



In [216]:
treea = Tree('s', a)
treea.pretty_print()

     s                      
  ___|___________            
 s       [('dp', [('d', ['  
         the']), ('np', ['  
        dog'])]), ('vp', [('
        v', ['chased']), (' 
           dp', [('d', ['   
         the']), ('np', ['  
            cat'])])])]     



In [217]:
tree.draw()

### tree element 추출
1. top -element "s"는 고려하지 않는다.
2. 원하는 위치에서 index에서 [,,....,0] 0을 추가한다.

In [94]:
tree[1,0,0]

'chased'

In [92]:
tree[1,1,0,0]

'the'

In [74]:
output_2.draw()
tree.draw()

### tree 안에 있는 element의 index 추출

In [142]:
# 위치 추출
tree.treepositions()

[(),
 (0,),
 (0, 0),
 (0, 0, 0),
 (0, 1),
 (0, 1, 0),
 (1,),
 (1, 0),
 (1, 0, 0),
 (1, 1),
 (1, 1, 0),
 (1, 1, 0, 0),
 (1, 1, 1),
 (1, 1, 1, 0)]

In [143]:
### meta charater find all == \\ 뒤에 쓰는 것

for i in tree.treepositions():
    #print(type(tree[list(i)]))
    if re.findall("\\(",str(tree[list(i)])) == []:
        print(i)
        print(tree[list(i)])


(0, 0, 0)
the
(0, 1, 0)
dog
(1, 0, 0)
chased
(1, 1, 0, 0)
the
(1, 1, 1, 0)
cat


In [104]:
tree.treeposition_spanning_leaves

<bound method Tree.treeposition_spanning_leaves of Tree('s', [Tree('dp', [Tree('d', ['the']), Tree('np', ['dog'])]), Tree('vp', [Tree('v', ['chased']), Tree('dp', [Tree('d', ['the']), Tree('np', ['cat'])])])])>

## 이제 tree 함수 이용해서 -> 한국어 적용

In [182]:
words = konlpy.tag.Twitter().pos("TV 테스트 겸 넷 플릭스 재생 해봤는데 액션 신에서 거슬리지 않으면서 적당히 심장에 쿵쿵 울림을 주는 소리가 나옵니다.5.")

C:\Users\Samsung\anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [183]:
chunker = RegexpParser(grammer_v2) 
output = chunker.parse(words)
tree = Tree("s",output)

In [184]:
print(tree)

(s
  TV/Alpha
  테스트/Noun
  겸/Noun
  넷/Noun
  플릭스/Noun
  재생/Noun
  (VP (V 해봤는데/Verb))
  액션/Noun
  신/Noun
  에서/Josa
  (VP (V 거슬리지/Verb))
  (VP (V 않으면서/Verb))
  적당히/Adjective
  심장/Noun
  에/Josa
  쿵쿵/Adverb
  울림/Noun
  을/Josa
  (VP (V 주는/Verb))
  소리/Noun
  가/Josa
  (VP (V 나옵니다/Verb))
  ./Punctuation
  5/Number
  ./Punctuation)


In [185]:
tree.pretty_print()

                                                                                                         s                                                                                                                       
    _____________________________________________________________________________________________________|__________________________________________________________________________________________________________________      
   |        |       |      |       |        |       |      |       |          |          |      |        |        |      |       |      |          |          |           |           VP        VP        VP       VP       VP   
   |        |       |      |       |        |       |      |       |          |          |      |        |        |      |       |      |          |          |           |           |         |         |        |        |     
   |        |       |      |       |        |       |      |       |          |          |    

In [189]:
for i in tree.treepositions():
    print(tree[list(i)][0])
    
    print("_"*50)

('TV', 'Alpha')
__________________________________________________
TV
__________________________________________________
테스트
__________________________________________________
겸
__________________________________________________
넷
__________________________________________________
플릭스
__________________________________________________
재생
__________________________________________________
(V 해봤는데/Verb)
__________________________________________________
('해봤는데', 'Verb')
__________________________________________________
해봤는데
__________________________________________________
액션
__________________________________________________
신
__________________________________________________
에서
__________________________________________________
(V 거슬리지/Verb)
__________________________________________________
('거슬리지', 'Verb')
__________________________________________________
거슬리지
__________________________________________________
(V 않으면서/Verb)
__________________________________________________
('않으면

In [198]:
for i in tree.treepositions():
    #print(type(tree[list(i)]))
    #print(str(tree[list(i)]))
    if re.findall("\\(",str((tree[list(i)])[0])) == []:
        print(i)
        print(tree[list(i)])


(0,)
('TV', 'Alpha')
(1,)
('테스트', 'Noun')
(2,)
('겸', 'Noun')
(3,)
('넷', 'Noun')
(4,)
('플릭스', 'Noun')
(5,)
('재생', 'Noun')
(6, 0, 0)
('해봤는데', 'Verb')
(7,)
('액션', 'Noun')
(8,)
('신', 'Noun')
(9,)
('에서', 'Josa')
(10, 0, 0)
('거슬리지', 'Verb')
(11, 0, 0)
('않으면서', 'Verb')
(12,)
('적당히', 'Adjective')
(13,)
('심장', 'Noun')
(14,)
('에', 'Josa')
(15,)
('쿵쿵', 'Adverb')
(16,)
('울림', 'Noun')
(17,)
('을', 'Josa')
(18, 0, 0)
('주는', 'Verb')
(19,)
('소리', 'Noun')
(20,)
('가', 'Josa')
(21, 0, 0)
('나옵니다', 'Verb')
(22,)
('.', 'Punctuation')
(23,)
('5', 'Number')
(24,)
('.', 'Punctuation')


In [202]:
tree

The Ghostscript executable isn't found.
See http://web.mit.edu/ghostscript/www/Install.htm
If you're using a Mac, you can try installing
https://docs.brew.sh/Installation then `brew install ghostscript`


LookupError: 

Tree('s', [('TV', 'Alpha'), ('테스트', 'Noun'), ('겸', 'Noun'), ('넷', 'Noun'), ('플릭스', 'Noun'), ('재생', 'Noun'), Tree('VP', [Tree('V', [('해봤는데', 'Verb')])]), ('액션', 'Noun'), ('신', 'Noun'), ('에서', 'Josa'), Tree('VP', [Tree('V', [('거슬리지', 'Verb')])]), Tree('VP', [Tree('V', [('않으면서', 'Verb')])]), ('적당히', 'Adjective'), ('심장', 'Noun'), ('에', 'Josa'), ('쿵쿵', 'Adverb'), ('울림', 'Noun'), ('을', 'Josa'), Tree('VP', [Tree('V', [('주는', 'Verb')])]), ('소리', 'Noun'), ('가', 'Josa'), Tree('VP', [Tree('V', [('나옵니다', 'Verb')])]), ('.', 'Punctuation'), ('5', 'Number'), ('.', 'Punctuation')])

### 저장해보기

In [152]:
cf = CanvasFrame()
tc = TreeWidget(cf.canvas(),tree)
cf.add_widget(tc,10,10) # (10,10) offsets
cf.print_to_file('tree_1.ps')

In [151]:
os.system('convert output.ps output.png')

4

In [154]:
cf

In [155]:
cf.draw()

AttributeError: 'CanvasFrame' object has no attribute 'draw'

In [177]:
tree.draw()

In [170]:
TreeView(tree)._cframe.print_to_file(r'C:\Users\Samsung\Desktop\all\img\7\output_4.ps')

In [159]:
os.path

<module 'ntpath' from 'C:\\Users\\Samsung\\anaconda3\\lib\\ntpath.py'>

In [167]:
a= tree.draw()

In [172]:
#export_png(a, filename=r'C:\Users\Samsung\Desktop\all\img\7\output_2.png')

In [176]:
TreeView(tree)._cframe.print_to_file(r'C:\Users\Samsung\Desktop\Project\이희정 교수님\project\parse_tree\test\output_5.jpg')

In [175]:
def file_converter(file):
    os.system("file_converter.py {0} > {1}".format(file, file.replace(".ps", ".jpg")))

base_dir = r"C:\Users\Samsung\Desktop\Project\이희정 교수님\project\parse_tree\test"

for file in os.listdir(base_dir):
    threading.Thread(target=file_converter, args=(file,)).start()